In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

In [2]:
# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 동영상 파일 열기
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\N\N\00002_H_A_N_C1\00002_H_A_N_C1.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: 비디오 파일을 열 수 없습니다.")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_수동bbox_2_+-100.mp4', fourcc, 60.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 종료")
        break

    frame_count += 1
    print(f"프레임 {frame_count} 처리 중")

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 사람 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                x1 = x1 - 100
                y1 = y1 - 100
                x2 = x2 + 100
                y2 = y2 + 100
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    print("포즈 랜드마크 감지됨")
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(frame, (global_x, global_y), 5, (0, 0, 255), 5)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                
                # 바운딩 박스 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 5)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", frame)
    
    # 결과 동영상 저장
    out.write(frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("프로그램 종료")

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공
프레임 1 처리 중

0: 384x640 1 Non_Fall, 71.3ms
Speed: 4.0ms preprocess, 71.3ms inference, 38.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (949, 719, 1898, 2255)
포즈 랜드마크 감지됨
랜드마크 0: (1292, 907)
랜드마크 11: (1436, 999)
랜드마크 12: (1191, 1059)
랜드마크 15: (1404, 1074)
랜드마크 16: (1162, 1106)
랜드마크 23: (1460, 1340)
랜드마크 24: (1293, 1379)
랜드마크 25: (1580, 1533)
랜드마크 26: (1248, 1642)
랜드마크 27: (1642, 1856)
랜드마크 28: (1259, 1966)
프레임 2 처리 중

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (949, 719, 1898, 2255)
포즈 랜드마크 감지됨
랜드마크 0: (1292, 908)
랜드마크 11: (1434, 1014)
랜드마크 12: (1180, 1065)
랜드마크 15: (1406, 1073)
랜드마크 16: (1161, 1106)
랜드마크 23: (1457, 1324)
랜드마크 24: (1269, 1358)
랜드마크 25: (1578, 1514)
랜드마크 26: (1228, 1619)
랜드마크 27: (1639, 1856)
랜드마크 28: (1266, 1960)
프레임 3 처리 중



c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0: 384x640 1 Non_Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (949, 719, 1898, 2255)
포즈 랜드마크 감지됨
랜드마크 0: (1291, 917)
랜드마크 11: (1433, 1016)
랜드마크 12: (1181, 1070)
랜드마크 15: (1409, 1073)
랜드마크 16: (1158, 1107)
랜드마크 23: (1457, 1325)
랜드마크 24: (1268, 1360)
랜드마크 25: (1601, 1497)
랜드마크 26: (1223, 1618)
랜드마크 27: (1637, 1862)
랜드마크 28: (1268, 1961)
프레임 4 처리 중

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (949, 719, 1898, 2255)
포즈 랜드마크 감지됨
랜드마크 0: (1294, 925)
랜드마크 11: (1433, 1017)
랜드마크 12: (1177, 1072)
랜드마크 15: (1414, 1073)
랜드마크 16: (1158, 1107)
랜드마크 23: (1457, 1324)
랜드마크 24: (1261, 1361)
랜드마크 25: (1600, 1477)
랜드마크 26: (1207, 1597)
랜드마크 27: (1637, 1864)
랜드마크 28: (1270, 1961)
프레임 5 처리 중

0: 384x640 1 Non_Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 2.0ms postprocess per image a

In [3]:
# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 동영상 파일 열기
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\N\N\00002_H_A_N_C3\00002_H_A_N_C3.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("비디오 파일 열기 실패")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
output_path = 'output_plot_bbox_5.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("열기 실패")
        break

    frame_count += 1

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    # YOLO 결과 시각화
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 클래스 0 : person 
                x1, y1, x2, y2 = map(int, box.xyxy[0])
             
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")

                # 바운딩 박스 추출
                person_image = frame[y1 : y2, x1 : x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 0, 255), 5)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                    
    out.write(annotated_frame)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    out.write(frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1916, 697, 2800, 1001)
포즈 랜드마크를 감지하지 못함

0: 384x640 1 Non_Fall, 3.0ms
Speed: 1.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1916, 697, 2800, 1001)
포즈 랜드마크를 감지하지 못함

0: 384x640 1 Non_Fall, 3.0ms
Speed: 2.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1916, 697, 2800, 1001)
랜드마크 0: (2643, 897)
랜드마크 11: (2639, 890)
랜드마크 12: (2645, 894)
랜드마크 15: (2647, 891)
랜드마크 16: (2651, 894)
랜드마크 23: (2653, 862)
랜드마크 24: (2657, 864)
랜드마크 25: (2668, 855)
랜드마크 26: (2678, 860)
랜드마크 27: (2686, 838)
랜드마크 28: (2691, 839)

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
YOL

In [4]:
# MacOS

import cv2
import mediapipe as mp
from ultralytics import YOLO

# 색상 정의
COLOR_DICT = {
    'person': (0, 255, 0),  
    'Fall': (0, 0, 255),    
    'Non_Fall': (255, 0, 0)   
}

# 라벨에 따른 색상 선택
def get_color(class_name):
    if class_name in COLOR_DICT:
        return COLOR_DICT[class_name]
    else:
        print(f"Warning: Unknown class '{class_name}'. Using default color.")
        return (0, 255, 255)
    
def adjust_bbox(bbox, scale_factor, frame_shape):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    new_x1 = max(0, int(center_x - new_width / 2))
    new_y1 = max(0, int(center_y - new_height / 2))
    new_x2 = min(int(center_x + new_width / 2), frame_shape[1])
    new_y2 = min(int(center_y + new_height / 2), frame_shape[0])
    
    return [new_x1, new_y1, new_x2, new_y2]

# YOLO 모델 로드
model = YOLO('path/to/your/best.pt') 

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 웹캠 실행
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: 웹캠을 열 수 없습니다.")
    exit()
print("웹캠 열기 성공")

while True:
    success, frame = cap.read()
    if not success:
        print("웹캠에서 프레임을 읽을 수 없습니다.")
        break

    # YOLO로 객체 감지
    results = model(frame)

    # 원본 프레임 복사
    annotated_frame = frame.copy()

    # YOLO 결과 처리 부분
    for r in results:
        boxes = r.boxes
        for box in boxes:
            class_name = model.names[int(box.cls)]
            print(f'class_name: {class_name}')
            
            # 원본 바운딩 박스 좌표 추출
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # 바운딩 박스 크기 조절(20% 확대)
            adjusted_bbox = adjust_bbox([x1, y1, x2, y2], scale_factor=1.2, frame_shape=frame.shape)
            
            # 라벨 생성
            label = f"{class_name} {box.conf.item():.2f}"
            
            # 라벨에 따른 색상 선택
            color = get_color(class_name)
            print(f"Class: {class_name}, Selected color: {color}")
            
            # 조절된 바운딩 박스 그리기
            cv2.rectangle(annotated_frame, (adjusted_bbox[0], adjusted_bbox[1]), 
                          (adjusted_bbox[2], adjusted_bbox[3]), color, 3)
            
            # 라벨 추가
            cv2.putText(annotated_frame, label, (adjusted_bbox[0], adjusted_bbox[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
            
            # 조절된 바운딩 박스에서 person_image 추출
            person_image = frame[adjusted_bbox[1]:adjusted_bbox[3], adjusted_bbox[0]:adjusted_bbox[2]]
            
            # MediaPipe로 포즈 추정
            results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
            if results_pose.pose_landmarks:
                for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                    if idx in DESIRED_LANDMARKS:
                        cx = int(landmark.x * person_image.shape[1])
                        cy = int(landmark.y * person_image.shape[0])
                        global_x = adjusted_bbox[0] + cx
                        global_y = adjusted_bbox[1] + cy
                        cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 0, 255), 5)
                        print(f"랜드마크 {idx}: ({global_x}, {global_y})")
            else:
                print("포즈 랜드마크를 감지하지 못함")
                
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'path\\to\\your\\best.pt'

In [5]:
# bbox 조절 함수
def adjust_bbox(bbox, scale_factor):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    new_x1 = max(0, int(center_x - new_width / 2))
    new_y1 = max(0, int(center_y - new_height / 2))
    new_x2 = min(int(center_x + new_width / 2), frame.shape[1])
    new_y2 = min(int(center_y + new_height / 2), frame.shape[0])
    
    return [new_x1, new_y1, new_x2, new_y2]

# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# Mediapipe 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# video path
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\N\N\00002_H_A_N_C3\00002_H_A_N_C3.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("비디오 열기 실패")
    exit()
print("비디오 파일 열기 성공")

# 결과 저장을 위한 설정
output_path = 'output_plot_bbox_adjusted_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("실행 실패")
        break

    frame_count += 1

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    # 결과 시각화
    annotated_frame = results[0].plot()

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 클래스 0 : person
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # 바운딩 박스 크기 조절 (scale_factor = 1.2 : 20% 확대)
                adjusted_bbox = adjust_bbox([x1, y1, x2, y2], scale_factor=1.2)
                x1, y1, x2, y2 = adjusted_bbox
                
                print(f"사람 감지: 조절된 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 조절된 바운딩 박스 그리기
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 0), 2)
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 0, 255), 5)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                    
    out.write(annotated_frame)

    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공

0: 384x640 1 Non_Fall, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 조절된 바운딩 박스 좌표 (1827, 666, 2888, 1031)
랜드마크 0: (2673, 824)
랜드마크 11: (2547, 912)
랜드마크 12: (2599, 857)
랜드마크 15: (2413, 794)
랜드마크 16: (2441, 797)
랜드마크 23: (2334, 879)
랜드마크 24: (2368, 857)
랜드마크 25: (2208, 800)
랜드마크 26: (2218, 793)
랜드마크 27: (2066, 770)
랜드마크 28: (2080, 789)

0: 384x640 1 Non_Fall, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 조절된 바운딩 박스 좌표 (1827, 666, 2888, 1031)
랜드마크 0: (2667, 826)
랜드마크 11: (2537, 935)
랜드마크 12: (2599, 852)
랜드마크 15: (2417, 807)
랜드마크 16: (2449, 790)
랜드마크 23: (2315, 875)
랜드마크 24: (2348, 830)
랜드마크 25: (2162, 822)
랜드마크 26: (2201, 788)
랜드마크 27: (2033, 785)
랜드마크 28: (2080, 781)

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 

In [15]:

# 색상 정의
COLOR_DICT = {
    'person': (0, 255, 0),  
    'Fall': (0, 0, 255),    
    'Non_Fall': (255, 0, 0)   
}

# 라벨에 따른 색상 선택
def get_color(class_name):
    if class_name in COLOR_DICT:
        return COLOR_DICT[class_name]
    else:
        print(f"Warning: Unknown class '{class_name}'. Using default color.")
        return (0, 255, 255)
    
def adjust_bbox(bbox, scale_factor, frame_shape):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    new_x1 = max(0, int(center_x - new_width / 2))
    new_y1 = max(0, int(center_y - new_height / 2))
    new_x2 = min(int(center_x + new_width / 2), frame_shape[1])
    new_y2 = min(int(center_y + new_height / 2), frame_shape[0])
    
    return [new_x1, new_y1, new_x2, new_y2]


# YOLO 모델 로드
model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 비디오 캡처 설정
cap = cv2.VideoCapture(r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Validation\01.원천데이터\VS\영상\N\N\01631_O_E_N_C2\01631_O_E_N_C2.mp4')

if not cap.isOpened():
    print("Error: 비디오 파일을 열 수 없습니다.")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
output_path = 'output_plot_bbox_adjusted_10.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("비디오 종료")
        break

    # YOLO로 객체 감지
    results = model(frame)

    # 원본 프레임 복사
    annotated_frame = frame.copy()

    # YOLO 결과 처리 부분
    for r in results:
        boxes = r.boxes
        for box in boxes:
            class_name = model.names[int(box.cls)]
            print(f'class_name " {class_name}')
            
            # 원본 바운딩 박스 좌표 추출
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # 바운딩 박스 크기 조절(20% 확대)
            adjusted_bbox = adjust_bbox([x1, y1, x2, y2], scale_factor=1.2, frame_shape=frame.shape)
            
            # 라벨 생성
            label = f"{model.names[int(box.cls)]} {box.conf.item():.2f}"
            
            # 라벨에 따른 색상 선택
            color = get_color(class_name)
            print(f"Class: {class_name}, Selected color: {color}")
            
            # 조절된 바운딩 박스 그리기
            cv2.rectangle(annotated_frame, (adjusted_bbox[0], adjusted_bbox[1]), 
                          (adjusted_bbox[2], adjusted_bbox[3]), color, 3)
            
            # 라벨 추가
            cv2.putText(annotated_frame, label, (adjusted_bbox[0], adjusted_bbox[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
            
            # 조절된 바운딩 박스에서 person_image 추출
            person_image = frame[adjusted_bbox[1]:adjusted_bbox[3], adjusted_bbox[0]:adjusted_bbox[2]]
            
            # MediaPipe로 포즈 추정
            results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
            if results_pose.pose_landmarks:
                print("포즈 랜드마크 감지됨")
                for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                    if idx in DESIRED_LANDMARKS:
                        cx = int(landmark.x * person_image.shape[1])
                        cy = int(landmark.y * person_image.shape[0])
                        global_x = adjusted_bbox[0] + cx
                        global_y = adjusted_bbox[1] + cy
                        cv2.circle(annotated_frame, (global_x, global_y), 5, (0, 0, 255), 5)
                        print(f"랜드마크 {idx}: ({global_x}, {global_y})")
            else:
                print("포즈 랜드마크를 감지하지 못함")
    
    out.write(annotated_frame)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", annotated_frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("프로그램 종료")

MediaPipe 설정 완료
비디오 파일 열기 성공

0: 384x640 1 Non_Fall, 4.0ms
Speed: 2.0ms preprocess, 4.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
class_name " Non_Fall
Class: Non_Fall, Selected color: (255, 0, 0)
포즈 랜드마크 감지됨
랜드마크 0: (2118, 976)
랜드마크 11: (1962, 910)
랜드마크 12: (2080, 1040)
랜드마크 15: (2004, 973)
랜드마크 16: (2050, 1119)
랜드마크 23: (1758, 1133)
랜드마크 24: (1790, 1218)
랜드마크 25: (1994, 1141)
랜드마크 26: (2037, 1275)
랜드마크 27: (1891, 1299)
랜드마크 28: (1920, 1456)

0: 384x640 1 Non_Fall, 3.0ms
Speed: 2.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
class_name " Non_Fall
Class: Non_Fall, Selected color: (255, 0, 0)
포즈 랜드마크 감지됨
랜드마크 0: (2105, 979)
랜드마크 11: (1960, 900)
랜드마크 12: (2047, 1042)
랜드마크 15: (2006, 994)
랜드마크 16: (2037, 1124)
랜드마크 23: (1733, 1157)
랜드마크 24: (1781, 1232)
랜드마크 25: (1978, 1143)
랜드마크 26: (2038, 1269)
랜드마크 27: (1866, 1306)
랜드마크 28: (1920, 1454)

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postpro